In [20]:
import json
import copy
import numpy as np

In [221]:
with open('test/annots/annotations_3less1st.json') as json_file:
    annots = json.load(json_file)

shapes = copy.deepcopy(annots[0]['shapes'])
# filter shapes
filtered_shapes = [shape for shape in shapes if shape['type'] ==
                   'polygon' and shape['label'] in ['lungslidingpresent', 'lungslidingabsent']]
# group shapes by frame
frame = 0
last_frame = filtered_shapes[-1]['frame']
lines_by_frame = []
lines_in_frame = []
for line in filtered_shapes:
    if line['frame'] == frame:
        lines_in_frame.append(line)
    else:
        frame = frame + 10 if frame < last_frame else last_frame
        lines_by_frame.append(lines_in_frame)
        lines_in_frame = []
        lines_in_frame.append(line)

In [222]:
num_of_lines = len(filtered_shapes)
len(filtered_shapes)

25

In [210]:
filtered_shapes[-1]['frame']

222

In [211]:
filtered_shapes[0]

{'type': 'polygon',
 'occluded': False,
 'z_order': 0,
 'points': [142.08984375,
  274.716796875,
  200.41312272174946,
  322.5419198055897,
  298.9793438639117,
  321.95868772782524,
  363.71810449574696,
  342.3718104495747,
  388.7970838396104,
  333.0400972053467,
  356.71931956257504,
  313.79343863912436,
  234.82381530984094,
  303.2952612393674,
  195.74726609963545,
  298.6294046172534],
 'frame': 0,
 'group': 0,
 'source': 'manual',
 'attributes': [],
 'label': 'lungslidingpresent'}

In [215]:
print(len(lines_by_frame))
print(len(lines_by_frame[0][0]['points']))
print(len(lines_by_frame[1][0]['points']))

23
16
22


In [214]:
skelet = {
    "type": "polygon",
    "occluded": False,
    "z_order": 0,
    "points": [],
    "frame": 0,
    "group": 0,
    "source": "manual",
    "attributes": [],
    "label": "lungslidingpresent"
}

In [217]:
indicies = dict((d['frame'], index) for (index, d) in enumerate(shapes))
print(indicies)

{0: 1, 10: 2, 20: 3, 30: 4, 40: 5, 50: 6, 60: 7, 70: 8, 80: 9, 90: 10, 100: 11, 110: 12, 120: 13, 130: 14, 140: 15, 150: 16, 160: 17, 170: 18, 180: 19, 190: 20, 200: 21, 210: 22, 220: 24, 222: 25}


In [218]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield np.array(lst[i:i + n])

In [219]:
def add_substract_point(shapes: list[list[float]]):
    upper_left_idx = []
    add = True if len(shapes[0]) < len(shapes[1]) else False
    coords = [list(chunks(shapes[0], 2)),
              list(chunks(shapes[1], 2))]
    for i in range(len(shapes)):
        magnitudes = [np.sqrt(x.dot(x)) for x in coords[i]]
        # get most upper left
        upper_left = min(magnitudes)
        # append index of the upper left coord to list of max 2 indicies
        upper_left_idx.append(magnitudes.index(upper_left))

    # value by which all points will be shifted
    shift = coords[1][upper_left_idx[1]] - coords[0][upper_left_idx[0]]
    # shift all points in the second shape - align polygons
    # shifted_polygon = [point - shift for point in coords[1]]
    # start polygons from the most upper left point
    if upper_left_idx[0] != 0:
        coords[0] = np.roll(coords[0], -upper_left_idx[0], axis=0)
    if upper_left_idx[1] != 0:
        coords[1] = np.roll(coords[1], -upper_left_idx[1], axis=0)
    # number of points to be added or substracted
    diff = abs(len(coords[0]) - len(coords[1]))
    # count distances between all points
    distances_in_first = [int(np.linalg.norm(
        coords[0][i] - coords[0][i+1])) for i in range(len(coords[0])-1)]
    distances_in_second = [int(np.linalg.norm(
        coords[1][i] - coords[1][i+1])) for i in range(len(coords[1])-1)]

    for _ in range(diff):
        for i in range(len(distances_in_first)):
            if abs(distances_in_first[i] - distances_in_second[i]) > 10:
                if add:
                    coords[0] = np.insert(
                        coords[0], i+1, (coords[1][i+1] - shift), axis=0)                    
                else:
                    coords[0] = np.delete(coords[0], i+1, axis=0)
                break

    return coords[0].flatten().tolist()

In [223]:
add_substract_point([lines_by_frame[0][0]['points'], lines_by_frame[1][0]['points']])

[142.08984375,
 274.716796875,
 136.84082624544317,
 287.54799513973194,
 136.84082624544317,
 287.54799513973194,
 136.84082624544317,
 287.54799513973194,
 200.41312272174946,
 322.5419198055897,
 298.9793438639117,
 321.95868772782524,
 363.71810449574696,
 342.3718104495747,
 388.7970838396104,
 333.0400972053467,
 356.71931956257504,
 313.79343863912436,
 234.82381530984094,
 303.2952612393674,
 195.74726609963545,
 298.6294046172534]

In [168]:
def interpolator(frames: list[dict]):
    """
    Interpolates between two frames
    If polygons have different number of vertices, it adds or substracts vertices

        Parameters:
            frames (list[dict]): shapes grouped by frames which they are present in
    """
    offset = 1
    # iterates over grouped by frame shapes
    for i, group in enumerate(frames[:-1]):
        cut = max(0, len(group) - len(frames[i+1]))
        frames_to_interpolate = frames[i+1][0]['frame'] - frames[i][0]['frame']
        difference, step = [], []
        # iterates over shapes in a group
        for j, shape in enumerate(group):
            # TODO: One less in 1st
            # add points to the shape if the next shape has less points
            if len(shape['points']) < len(frames[i+1][j]['points']):
                shape['points'] = add_substract_point(
                    [shape['points'], frames[i+1][j]['points']])
            # get the difference between the points of the same shape in the next group
            difference = (np.subtract(
                frames[i+1][j]['points'], shape['points']))
            # get the step by which the points will be interpolated
            step.append(difference / frames_to_interpolate)
        # iterates over each shape in a group
        for k in range(1, frames_to_interpolate):
            for j, shape in enumerate(group[:-cut] if cut > 0 else group):
                points = shape['points'].copy() + (step[j] * k)
                skelet['points'] = list(points)
                skelet['frame'] = shape['frame'] + k
                annots[0]['shapes'].insert(indicies.get(
                    shape['frame']) + offset, copy.deepcopy(skelet))
                offset += 1

In [178]:
interpolator(lines_by_frame)

In [21]:
# with open('annotations.json') as json_file:
#     annots = json.load(json_file)

In [170]:
with open('test/annots/annotations_less1st->fixed.json', 'w') as aline_file:
    json.dump(annots, aline_file)